In [13]:
import tensorflow as tf
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

print(tf.__version__)

2.4.1


# Section 5.2 in the book 'Deeplearning with Python" by Francois Challet

  1. CONVOLUTION LAYER TO CLASSIFY DOGS VS CATS.
  2. DATA AUGMENTATION TECHNIQUE TO OVERCOME OVER-FITTING.
  
  CATS - DOGS DATA HAS TO BE TAKEN FROM : https://www.kaggle.com/c/dogs-vs-cats \
  data/train/cats - 1000 cat images \
  data/train/dogs - 1000 dog images \
  data/validation/cats - 500 cat images \
  data/validation/dogs - 500 dog images \
  data/test/cats - 500 cat images \
  data/test/dogs - 500 dog images

In [14]:
import os, shutil

base_dir = './DATA/CATS_DOGS'

train_dir = os.path.join(base_dir, 'train')
train_cat_dir = os.path.join(train_dir, 'cats')
train_dog_dir = os.path.join(train_dir, 'dogs')
test_dir = os.path.join(base_dir, 'test')
test_cat_dir = os.path.join(test_dir, 'cats')
test_dog_dir = os.path.join(test_dir, 'dogs')
val_dir = os.path.join(base_dir, 'validation')
val_cat_dir = os.path.join(val_dir, 'cats')
val_dog_dir = os.path.join(val_dir, 'dogs')

print('Total CAT training images ', len(os.listdir(train_cat_dir)))
print('Total DOG training images ' , len(os.listdir(train_dog_dir)))
print('Total CAT test images ', len(os.listdir(test_cat_dir)))
print('Total DOG test images ' , len(os.listdir(test_dog_dir)))
print('Total CAT validation images ' , len(os.listdir(val_cat_dir)))
print('Total DOG training images ', len(os.listdir(val_dog_dir)))

Total CAT training images  1000
Total DOG training images  1000
Total CAT test images  500
Total DOG test images  500
Total CAT validation images  500
Total DOG training images  500


### 1. CLASSIFICATION CONVOLUTIONAL NETWORK

   The Depth of the feature maps progressively increases in the network (from 32 to 128) \
   Where as the size of the feature maps decreases (from 148 X 148 X 7 X 7).
   

In [15]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

### 2. Configuring the model for Training

    1. We shall use the RMSprop Optimizer. 
    2. We shall use binary cross entropy as the loss.
    3. And the learning rate is (lr = 1e-4)
    

In [16]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])


### 3. DATA PREPROCESSING OF THE IMAGE FILES.

    1. Read the Images Files.
    2. Decode the JPEG content to RGB grids of pixels.
    3. Convert these into floating-point tensors.
    4. Rescale the pixel values (between 0 to 255) to [0, 1] interval.
    
    All the above functionalities are done by the ImageDataGenerator.

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), batch_size=20,
                                                    class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size = (150, 150), batch_size=20,
                                                    class_mode='binary')

for data_batch, labels_batch in train_generator:
    print('data batch shape ', data_batch.shape)
    print('data labels batch shape ', labels_batch.shape)
    break;

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
data batch shape  (20, 150, 150, 3)
data labels batch shape  (20,)


### 4. Running the Model. 
    
    Saving the Model using the model.sava function.
    One thing to remember is that when we give the steps_per_epoch 
    and validation_steps, which tells the process how many batches to draw from the
    train_generator and validation_generator

In [19]:
history = model.fit_generator(
                train_generator, steps_per_epoch=100, epochs=30, 
                validation_data = val_generator, validation_steps=50)

model.save('cats_dogs_small_1.h5')

Epoch 1/30
 46/100 [============>.................] - ETA: 31s - loss: 0.6781 - acc: 0.5750

UnknownError:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fa9c5f067c0>
Traceback (most recent call last):

  File "/Users/narendra/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 249, in __call__
    ret = func(*args)

  File "/Users/narendra/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "/Users/narendra/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 891, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/narendra/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 807, in wrapped_generator
    for data in generator_fn():

  File "/Users/narendra/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/tensorflo